Predict the Length of Stay for each patient so that the hospitals can optimize resources and function better

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.set_printoptions(suppress=True)
import warnings
warnings.filterwarnings('ignore')

loading dataset

In [3]:
d1 = pd.read_csv('/content/sample_sub.csv')
d2 = pd.read_csv('/content/train_data_dictionary.csv')
test = pd.read_csv('/content/test_data.csv')
train = pd.read_csv('/content/train_data.csv')

Data exploration

In [4]:
train.head()

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397.0,7.0,Emergency,Extreme,2.0,51-60,4911.0,0-10
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397.0,7.0,Trauma,Extreme,2.0,51-60,5954.0,41-50
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397.0,7.0,Trauma,Extreme,2.0,51-60,4745.0,31-40
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397.0,7.0,Trauma,Extreme,2.0,51-60,7272.0,41-50
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397.0,7.0,Trauma,Extreme,2.0,51-60,5558.0,41-50


In [5]:
train.info()
train.Stay.unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50057 entries, 0 to 50056
Data columns (total 18 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   case_id                            50057 non-null  int64  
 1   Hospital_code                      50057 non-null  int64  
 2   Hospital_type_code                 50057 non-null  object 
 3   City_Code_Hospital                 50057 non-null  int64  
 4   Hospital_region_code               50057 non-null  object 
 5   Available Extra Rooms in Hospital  50057 non-null  int64  
 6   Department                         50057 non-null  object 
 7   Ward_Type                          50057 non-null  object 
 8   Ward_Facility_Code                 50057 non-null  object 
 9   Bed Grade                          50047 non-null  float64
 10  patientid                          50056 non-null  float64
 11  City_Code_Patient                  49601 non-null  flo

array(['0-10', '41-50', '31-40', '11-20', '51-60', '21-30', '71-80',
       'More than 100 Days', '81-90', '61-70', '91-100', nan],
      dtype=object)

In [6]:
train.isnull().sum().sort_values(ascending = False)

City_Code_Patient                    456
Bed Grade                             10
patientid                              1
Admission_Deposit                      1
Age                                    1
Visitors with Patient                  1
Severity of Illness                    1
Type of Admission                      1
Stay                                   1
Hospital_code                          0
Ward_Facility_Code                     0
Ward_Type                              0
Department                             0
Available Extra Rooms in Hospital      0
Hospital_region_code                   0
City_Code_Hospital                     0
Hospital_type_code                     0
case_id                                0
dtype: int64

In [7]:
train.shape

(50057, 18)

In [8]:
for i in train.columns:
    print(i, ':', train[i].nunique())

case_id : 50057
Hospital_code : 32
Hospital_type_code : 7
City_Code_Hospital : 11
Hospital_region_code : 3
Available Extra Rooms in Hospital : 13
Department : 5
Ward_Type : 6
Ward_Facility_Code : 6
Bed Grade : 4
patientid : 10029
City_Code_Patient : 35
Type of Admission : 3
Severity of Illness : 3
Visitors with Patient : 25
Age : 10
Admission_Deposit : 5617
Stay : 11


Data Preparation 

In [9]:
train['Bed Grade'].fillna(train['Bed Grade'].mode()[0], inplace = True)
test['Bed Grade'].fillna(test['Bed Grade'].mode()[0], inplace = True)

In [10]:
train['City_Code_Patient'].fillna(train['City_Code_Patient'].mode()[0], inplace = True)
test['City_Code_Patient'].fillna(test['City_Code_Patient'].mode()[0], inplace = True)

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['Stay'] = le.fit_transform(train['Stay'].astype('str'))

In [12]:
train.head()

,case_id,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,1,8,c,3,Z,3,radiotherapy,R,F,2.0,31397.0,7.0,Emergency,Extreme,2.0,51-60,4911.0,0
1,2,2,c,5,Z,2,radiotherapy,S,F,2.0,31397.0,7.0,Trauma,Extreme,2.0,51-60,5954.0,4
2,3,10,e,1,X,2,anesthesia,S,E,2.0,31397.0,7.0,Trauma,Extreme,2.0,51-60,4745.0,3
3,4,26,b,2,Y,2,radiotherapy,R,D,2.0,31397.0,7.0,Trauma,Extreme,2.0,51-60,7272.0,4
4,5,26,b,2,Y,2,radiotherapy,S,D,2.0,31397.0,7.0,Trauma,Extreme,2.0,51-60,5558.0,4


In [13]:
test['Stay'] = -1
df = pd.concat([train, test])
df.shape

(103231, 18)

In [14]:
for i in ['Hospital_type_code', 'Hospital_region_code', 'Department',
          'Ward_Type', 'Ward_Facility_Code', 'Type of Admission', 'Severity of Illness', 'Age']:
    le = LabelEncoder()
    df[i] = le.fit_transform(df[i].astype(str))

In [15]:
train = df[df['Stay']!=-1]
test = df[df['Stay']==-1]

In [17]:
def get_countid_enocde(train, test, cols, name):
  temp = train.groupby(cols)['case_id'].count().reset_index().rename(columns = {'case_id': name})
  temp2 = test.groupby(cols)['case_id'].count().reset_index().rename(columns = {'case_id': name})
  train = pd.merge(train, temp, how='left', on= cols)
  test = pd.merge(test,temp2, how='left', on= cols)
  train[name] = train[name].astype('float')
  test[name] = test[name].astype('float')
  train[name].fillna(np.median(temp[name]), inplace = True)
  test[name].fillna(np.median(temp2[name]), inplace = True)
  return train, test

In [18]:
train, test = get_countid_enocde(train, test, ['patientid'], name = 'count_id_patient')
train, test = get_countid_enocde(train, test, 
                                 ['patientid', 'Hospital_region_code'], name = 'count_id_patient_hospitalCode')
train, test = get_countid_enocde(train, test, 
                                 ['patientid', 'Ward_Facility_Code'], name = 'count_id_patient_wardfacilityCode')

In [19]:
test1 = test.drop(['Stay', 'patientid', 'Hospital_region_code', 'Ward_Facility_Code'], axis =1)
train1 = train.drop(['case_id', 'patientid', 'Hospital_region_code', 'Ward_Facility_Code'], axis =1)

In [20]:
X1 = train1.drop('Stay', axis =1)
y1 = train1['Stay']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size =0.20, random_state =100)